In [7]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey,\
    Date, Float
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import ClauseElement

engine = create_engine('sqlite:///:memory:', echo=True)

In [8]:
Model = declarative_base()
Session = sessionmaker()
Session.configure(bind=engine)

class Category(Model):
    __tablename__ = 'categories'
    
    id = Column('id', Integer, primary_key=True)
    name = Column('name', String)
    code = Column('code', String)
    
    def __init__(self, id, name, code):
        self.id = id
        self.name = name
        self.code = code
        
    def __repr__(self):
        return '<Category ({id}, {name}, {code})>'.format(
            id=self.id, name=self.name, code=self.code)
    
    @property
    def pk(self):
        return self.id
    

class Goods(Model):
    __tablename__ = 'goods'
    
    id = Column('id', Integer, primary_key=True)
    name = Column('name', String)
    mech = Column('mech', String)
    img_link = Column('img_link', String)
    price_reg_min = Column('price_reg_min', Float)
    price_promo_min = Column('price_promo_min', Float)
    category = Column('category_id', ForeignKey('categories.id'))
    promo = Column('promo_id', ForeignKey('promo.id'))
    
    def __init__(self, id, name, price_reg_min, price_promo_min, mech=None, img_link=None):
        self.id = id
        self.name = name
        self.mech = mech
        self.img_link = img_link
        self.price_reg_min = price_reg_min
        self.price_promo_min = price_promo_min
        
    def __repr__(self):
        return '<Category ({id}, {name})>'.format(id=self.id, name=self.name)
    
    @property
    def pk(self):
        return self.id
    

class Promo(Model):
    __tablename__ = 'promo'
    
    id = Column('id', Integer, primary_key=True)
    name = Column('name', String)
    date_begin = Column('date_begin', Date)
    date_end = Column('date_end', Date)
    promo_type = Column('promo_type', String)
    description = Column('description', String)
    kind = Column('kind', String)
    expired_at = Column('expired_at', Integer)
    
    def __init__(self, id, date_begin, date_end, promo_type, description, kind, expired_at):
        self.id = id
        self.date_begin = date_begin
        self.date_end = date_end
        self.promo_type = promo_type
        self.description = description
        self.kind = kind
        self.expired_at = expired_at
        
    def __repr__(self):
        return '<Category ({id}, {name})>'.format(id=self.id, name=self.name)
    
    @property
    def pk(self):
        return self.id
    
Model.metadata.create_all(engine)

2019-12-12 00:15:23,930 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-12-12 00:15:23,932 INFO sqlalchemy.engine.base.Engine ()
2019-12-12 00:15:23,934 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-12-12 00:15:23,936 INFO sqlalchemy.engine.base.Engine ()
2019-12-12 00:15:23,939 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("categories")
2019-12-12 00:15:23,940 INFO sqlalchemy.engine.base.Engine ()
2019-12-12 00:15:23,942 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("categories")
2019-12-12 00:15:23,943 INFO sqlalchemy.engine.base.Engine ()
2019-12-12 00:15:23,944 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("goods")
2019-12-12 00:15:23,947 INFO sqlalchemy.engine.base.Engine ()
2019-12-12 00:15:23,949 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("goods")
2019-12-12 00:15:23,951 INFO sqlalchemy.engine.base.Engine ()
2019-12-12 00:15

In [20]:
def get_or_create(session, model, defaults=None, **kwargs):
    instance = session.query(model).filter_by(**kwargs).first()
    if instance:
        return instance, False
    else:
        params = dict((k, v) for k, v in kwargs.items() if not isinstance(v, ClauseElement))
        params.update(defaults or {})
        instance = model(**params)
        session.add(instance)
        return instance, True

In [10]:
import requests
import json
from datetime import datetime

class ResponseError(BaseException):
    """
    Class for response exceptions
    """

In [11]:
def get_page_result(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise ResponseError("Error {code}".format(code=response.status_code))
    return response.json()

In [17]:
def save_result(data):
    results = data.get('results', [])
    session = Session()
    for result in results:
        promo_data = result.get("promo", {})
        promo, _ = get_or_create(session, Promo, 
            date_begin=datetime.strptime(promo_data.get("date_begin"), "%Y-%m-%d").date(),
            date_end=datetime.strptime(promo_data.get("date_end"), "%Y-%m-%d").date(),
            promo_type=promo_data.get("promo_type"),
            description=promo_data.get("description"),
            kind=promo_data.get("kind"),
            expired_at=promo_data.get("expired_at")
        )
        
        goods, _ = get_or_create(Session, Goods,
            id=result.get("id"),
            name=result.get("name"),
            mech=result.get("mech"),
            img_link=result.get("img_link"),
            price_reg_min=result.get("price_reg_min"),
            price_promo_min=result.get("price_promo_min")
        )
        

In [ ]:
start_url = "https://5ka.ru/api/v2/special_offers/?records_per_page=20&page=1"
result = get_page_result(start_url)
save_result(result)

In [ ]:
next_page = result.get('next')

session = Session()
goods = session.query(Goods).all()
print(goods)